In [1]:
import gc
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau

from HW_base import evaluate_build, focus_build, plt_scatter
from HW_keras import hw_layer

In [2]:
fname = 'Keras_F5-E128-F60_Linear-LSTM128x4-FC-SELU-FC_Batch1000'

In [3]:
data_train_df = pd.read_csv('./Database/train.csv')
data_test_df = pd.read_csv('./Database/test.csv')

x_columns = ['R', 'C', 'time_step', 'u_in', 'u_out']
y_columns = ['pressure']

data_train = data_train_df[x_columns].values.astype(np.float32)
data_train = data_train.reshape(-1, 80, data_train.shape[-1])

target_train = data_train_df[y_columns].values.astype(np.float32)
target_train = target_train.reshape(-1, 80, target_train.shape[-1])

data_test = data_test_df[x_columns].values.astype(np.float32)
data_test = data_test.reshape(-1, 80, data_test.shape[-1])

del data_train_df
del data_test_df
gc.collect()

22

In [4]:
np.random.seed(121212)
data_idx = np.arange(len(data_train))
np.random.shuffle(data_idx)

train_index = data_idx[:int(len(data_idx)*0.8)]
valid_index = data_idx[int(len(data_idx)*0.8):]

x_train, y_train = data_train[train_index], target_train[train_index]
x_valid, y_valid = data_train[valid_index], target_train[valid_index]
x_test = data_test

del data_train
del data_test
gc.collect()

20

In [5]:
evaluate_list = [evaluate_build(x_test[..., i], 128) for i in range(x_test.shape[-1])]
evaluate_focus_list = []
for evaluate in evaluate_list:
    focus = 1 - (len(evaluate) - 1)/10
    if focus < 0.6:
        focus = 0.6
    evaluate_focus = focus_build(evaluate, focus)
    evaluate_focus_list.append(evaluate_focus)

evaluate_num:   3,focus:0.8000: 3it [00:00, 1719.45it/s]
evaluate_num:   3,focus:0.8000: 3it [00:00, 880.35it/s]
evaluate_num: 127,focus:0.6000: 127it [00:00, 458.94it/s]
evaluate_num:  99,focus:0.6000: 99it [00:00, 329.90it/s]
evaluate_num:   2,focus:0.9000: 2it [00:00, 988.41it/s]


In [6]:
model = Sequential([
    hw_layer(evaluate_focus_list, input_shape=(80,5), name='hw_layer'),
    Dense(128, use_bias=False),
    Bidirectional(LSTM(128, use_bias=True, return_sequences=True)),
    Bidirectional(LSTM(128, use_bias=True, return_sequences=True)),
    Bidirectional(LSTM(128, use_bias=True, return_sequences=True)),
    Bidirectional(LSTM(128, use_bias=True, return_sequences=True)),
    Dense(32, use_bias=False, activation='selu'),
    Dense(1, name='hw_embedding')
])

In [7]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), loss='mae')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hw_layer (hw_layer)          (None, 80, 234)           0         
_________________________________________________________________
dense (Dense)                (None, 80, 128)           29952     
_________________________________________________________________
bidirectional (Bidirectional (None, 80, 256)           263168    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 80, 256)           394240    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 80, 256)           394240    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 80, 256)           394240    
_________________________________________________________________
dense_1 (Dense)              (None, 80, 32)            8

In [8]:
callback_list = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5,  patience=8, min_lr=1e-7, verbose=True),
    EarlyStopping(monitor='val_loss', patience=32),
    ModelCheckpoint(f'models/{fname}_best.h5', monitor='val_loss', save_best_only=True, verbose=False),
    TensorBoard(log_dir=f'./Log/{fname}')
]

model.fit(x_train, y_train, epochs=1000, batch_size=1000,
            validation_data=(x_valid, y_valid), validation_batch_size=1000,
            callbacks=callback_list)

Epoch 1/1000
61/61 [==============================] - 33s 304ms/step - loss: 10.8460 - val_loss: 9.3622
Epoch 2/1000
61/61 [==============================] - 13s 211ms/step - loss: 8.5134 - val_loss: 5.5986
Epoch 3/1000
61/61 [==============================] - 13s 211ms/step - loss: 5.3703 - val_loss: 5.1452
Epoch 4/1000
61/61 [==============================] - 13s 213ms/step - loss: 5.1372 - val_loss: 5.0860
Epoch 5/1000
61/61 [==============================] - 13s 214ms/step - loss: 5.0801 - val_loss: 5.0370
Epoch 6/1000
61/61 [==============================] - 13s 214ms/step - loss: 5.0300 - val_loss: 4.9873
Epoch 7/1000
61/61 [==============================] - 13s 215ms/step - loss: 4.9979 - val_loss: 4.9315
Epoch 8/1000
61/61 [==============================] - 13s 215ms/step - loss: 4.9320 - val_loss: 4.8627
Epoch 9/1000
61/61 [==============================] - 13s 215ms/step - loss: 4.8740 - val_loss: 4.7693
Epoch 10/1000
61/61 [==============================] - 13s 215ms/step - 

Epoch 80/1000
61/61 [==============================] - 13s 217ms/step - loss: 1.9445 - val_loss: 1.9406
Epoch 81/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.9361 - val_loss: 1.8698
Epoch 82/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.8753 - val_loss: 1.8952
Epoch 83/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.9086 - val_loss: 1.9267
Epoch 84/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.8836 - val_loss: 1.8640
Epoch 85/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.8655 - val_loss: 1.8067
Epoch 86/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.8516 - val_loss: 1.8436
Epoch 87/1000
61/61 [==============================] - 13s 218ms/step - loss: 1.8212 - val_loss: 1.8313
Epoch 88/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.8026 - val_loss: 1.8374
Epoch 89/1000
61/61 [==============================] - 13s 216ms

61/61 [==============================] - 13s 215ms/step - loss: 1.3649 - val_loss: 1.3690
Epoch 158/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.3726 - val_loss: 1.3669
Epoch 159/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.3644 - val_loss: 1.3655
Epoch 160/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.3616 - val_loss: 1.3657
Epoch 161/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.3608 - val_loss: 1.3625
Epoch 162/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.3567 - val_loss: 1.3610
Epoch 163/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.3596 - val_loss: 1.3590
Epoch 164/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.3626 - val_loss: 1.3574
Epoch 165/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.3546 - val_loss: 1.3559
Epoch 166/1000
61/61 [==============================] - 13s 215ms/step

61/61 [==============================] - 13s 214ms/step - loss: 1.2781 - val_loss: 1.2776
Epoch 236/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.2697 - val_loss: 1.2772
Epoch 237/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.2685 - val_loss: 1.2762
Epoch 238/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.2771 - val_loss: 1.2741
Epoch 239/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.2694 - val_loss: 1.2712
Epoch 240/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.2710 - val_loss: 1.2706
Epoch 241/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.2703 - val_loss: 1.2703
Epoch 242/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.2695 - val_loss: 1.2687
Epoch 243/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.2609 - val_loss: 1.2695
Epoch 244/1000
61/61 [==============================] - 13s 215ms/step

61/61 [==============================] - 13s 216ms/step - loss: 1.2185 - val_loss: 1.2153
Epoch 314/1000
61/61 [==============================] - 13s 217ms/step - loss: 1.2200 - val_loss: 1.2147
Epoch 315/1000
61/61 [==============================] - 13s 217ms/step - loss: 1.2230 - val_loss: 1.2148
Epoch 316/1000
61/61 [==============================] - 13s 217ms/step - loss: 1.2163 - val_loss: 1.2181
Epoch 317/1000
61/61 [==============================] - 13s 217ms/step - loss: 1.2121 - val_loss: 1.2163
Epoch 318/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.2157 - val_loss: 1.2140
Epoch 319/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.2152 - val_loss: 1.2196
Epoch 320/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.2108 - val_loss: 1.2107
Epoch 321/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.2105 - val_loss: 1.2208
Epoch 322/1000
61/61 [==============================] - 13s 215ms/step

...

61/61 [==============================] - 13s 214ms/step - loss: 1.1052 - val_loss: 1.1096
Epoch 625/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.1065 - val_loss: 1.1096
Epoch 626/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.1051 - val_loss: 1.1098
Epoch 627/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.1028 - val_loss: 1.1099
Epoch 628/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.1072 - val_loss: 1.1087
Epoch 629/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.1065 - val_loss: 1.1087
Epoch 630/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.1076 - val_loss: 1.1132
Epoch 631/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.1094 - val_loss: 1.1095
Epoch 632/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.1063 - val_loss: 1.1080
Epoch 633/1000
61/61 [==============================] - 13s 213ms/step

61/61 [==============================] - 13s 215ms/step - loss: 1.0945 - val_loss: 1.0973
Epoch 703/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0895 - val_loss: 1.0923
Epoch 704/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0859 - val_loss: 1.0918
Epoch 705/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0860 - val_loss: 1.0965
Epoch 706/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0873 - val_loss: 1.0917
Epoch 707/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0863 - val_loss: 1.0915
Epoch 708/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0870 - val_loss: 1.0913
Epoch 709/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0853 - val_loss: 1.0909
Epoch 710/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0845 - val_loss: 1.0912
Epoch 711/1000
61/61 [==============================] - 13s 214ms/step

61/61 [==============================] - 13s 214ms/step - loss: 1.0748 - val_loss: 1.0759
Epoch 781/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0726 - val_loss: 1.0757
Epoch 782/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0724 - val_loss: 1.0752
Epoch 783/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0728 - val_loss: 1.0749
Epoch 784/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0698 - val_loss: 1.0751
Epoch 785/1000
61/61 [==============================] - 13s 213ms/step - loss: 1.0735 - val_loss: 1.0793
Epoch 786/1000
61/61 [==============================] - 13s 213ms/step - loss: 1.0695 - val_loss: 1.0743
Epoch 787/1000
61/61 [==============================] - 13s 213ms/step - loss: 1.0685 - val_loss: 1.0824
Epoch 788/1000
61/61 [==============================] - 13s 213ms/step - loss: 1.0764 - val_loss: 1.0738
Epoch 789/1000
61/61 [==============================] - 13s 214ms/step

61/61 [==============================] - 13s 215ms/step - loss: 1.0490 - val_loss: 1.0627
Epoch 859/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0559 - val_loss: 1.0594
Epoch 860/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0461 - val_loss: 1.0594
Epoch 861/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0556 - val_loss: 1.0624
Epoch 862/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0577 - val_loss: 1.0606
Epoch 863/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0564 - val_loss: 1.0584
Epoch 864/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0567 - val_loss: 1.0586
Epoch 865/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0595 - val_loss: 1.0590
Epoch 866/1000
61/61 [==============================] - 13s 214ms/step - loss: 1.0548 - val_loss: 1.0586
Epoch 867/1000
61/61 [==============================] - 13s 214ms/step

61/61 [==============================] - 13s 215ms/step - loss: 1.0383 - val_loss: 1.0443
Epoch 937/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0449 - val_loss: 1.0442
Epoch 938/1000
61/61 [==============================] - 13s 218ms/step - loss: 1.0380 - val_loss: 1.0439
Epoch 939/1000
61/61 [==============================] - 13s 216ms/step - loss: 1.0400 - val_loss: 1.0446
Epoch 940/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0403 - val_loss: 1.0432
Epoch 941/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0387 - val_loss: 1.0434
Epoch 942/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0413 - val_loss: 1.0479
Epoch 943/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0423 - val_loss: 1.0489
Epoch 944/1000
61/61 [==============================] - 13s 215ms/step - loss: 1.0393 - val_loss: 1.0463
Epoch 945/1000
61/61 [==============================] - 13s 215ms/step